# 🧠 SimAI Case Study – Product Specialist

This notebook covers the complete solution to the take-home assignment for the SimAI platform.

We address the following problems:
1. 3D data visualization
2. Projection of simulation values from volume to surface
3. Computation of aerodynamic drag
4. Supporting code and visual outputs for discussion

## 🎯 Goals
- Help a user visualize 3D simulation data.
- Project physical simulation values from the volume mesh to the surface mesh.
- Compute drag using projected fields.
- Present clear code and visualizations.

We use the `pyvista` library for mesh manipulation and plotting.

## 📦 Imports and Setup

In [ ]:
import pyvista as pv
import numpy as np
import os

# Ensure plots are rendered inline
pv.set_jupyter_backend('pythreejs')  # You can use 'panel' or 'static' as fallback

# Set data directory
DATA_DIR = os.path.join('src', 'data')

## 📁 Load Geometry and Simulation Files
We begin by loading the surface and volume files for both the car and the spoiler. These are provided in `.vtp` (surface) and `.vtp`/`.vtu` (volume) formats.

In [ ]:
# Car geometry
surface_car = pv.read(os.path.join(DATA_DIR, 'surface_car.vtp'))
volume_car = pv.read(os.path.join(DATA_DIR, 'volume_car.vtp'))

# Spoiler geometry
surface_spoiler = pv.read(os.path.join(DATA_DIR, 'surface_spoiler.vtp'))
volume_spoiler = pv.read(os.path.join(DATA_DIR, 'volume_spoiler.vtu'))

## 🔍 Visualize the 3D Meshes

We use PyVista's interactive 3D plotting to inspect the geometries. Below, we plot the surface of the car and the volume of the spoiler (colored by `pressure`).

In [ ]:
# Visualize surface of the car
surface_car.plot(title='Car Surface Mesh', show_edges=True)

In [ ]:
# Visualize pressure field on spoiler volume
volume_spoiler.plot(scalars='pressure', cmap='coolwarm', title='Spoiler Volume – Pressure Field')

## 📌 Project Volume Values onto the Surface
The surface mesh lacks the necessary `pressure` and `wall_shear` values to compute drag. These values are available in the volume mesh.

We use nearest-neighbor interpolation with `sample()` to project these fields onto the surface nodes.

In [ ]:
# Inspect available fields in volume
print('Volume scalar fields:', volume_car.array_names)

# Project fields to surface
interpolated_surface = surface_car.sample(volume_car)

### 💾 Save Surface with Projected Data

In [ ]:
# Save the new surface with added fields
output_path = os.path.join(DATA_DIR, 'surface_car_with_vars.vtp')
interpolated_surface.save(output_path)
print(f"Saved new surface with projected data to {output_path}")

### 🎨 Visualize Projected Pressure

In [ ]:
interpolated_surface.plot(scalars='pressure', cmap='coolwarm', title='Projected Pressure Field on Car Surface')

## 🧮 Compute Estimated Drag
Drag is defined as:
\[ \text{Drag} = \int_{S} (\text{pressure} + \text{wall\_shear}) \, dS \]

To approximate this, we:
- Sum the `pressure + wall_shear` across all surface nodes.
- Note: In a full CFD post-processing, you'd integrate over surface area, but we simplify here.

In [ ]:
# Ensure wall_shear is present (or fill with zeros)
pressure = interpolated_surface['pressure']
wall_shear = (interpolated_surface['wall_shear']
              if 'wall_shear' in interpolated_surface.array_names else np.zeros_like(pressure))

drag_estimate = np.sum(pressure + wall_shear)
print(f"Estimated drag (summed field values): {drag_estimate:.4f}")

## ✅ Conclusion
- We successfully visualized and inspected 3D geometry.
- We projected pressure and wall shear from the volume to the surface.
- We estimated aerodynamic drag using the projected fields.

This notebook can be used to help the user of SimAI understand how to preprocess data and interpret physical results using Python and PyVista.